In [1]:
import sqlite3
import pandas as pd
import json
import os
import matplotlib.pyplot as plt
from IPython.display import Markdown, display
import calendar


# Allow Markdown characters in the print statement
# this can make the output more print ready
def printmd(string):
    display(Markdown(string))

In [2]:
conn = sqlite3.connect("twitterTesla.db")
c = conn.cursor()

# Create tesla table to hold Elon Musk's tweets

sql_create = """
CREATE TABLE IF NOT EXISTS tweets (
    id_str TEXT PRIMARY KEY,
    created_at INTEGER,
    full_text TEXT
);
"""

# remove any data from a previous run
sql_delete = "DELETE FROM tweets"

try:
    c.execute(sql_create)
    c.execute(sql_delete)
    conn.commit()
finally:
    conn.rollback()
    c.close()
    conn.close()

In [3]:
conn = sqlite3.connect("twitterTesla.db")
c = conn.cursor()

# Create tesla table to hold stock prices

sql_create = """
CREATE TABLE IF NOT EXISTS tesla (
    id INTEGER PRIMARY KEY,
    date TEXT,
    open DECIMAL(6,2),
    close DECIMAL(6,2),
    high DECIMAL(6,2),
    low DECIMAL(6,2)
);
"""

# remove any data from a previous run
sql_delete = "DELETE FROM tesla"

try:
    c.execute(sql_create)
    c.execute(sql_delete)
    conn.commit()
finally:
    conn.rollback()
    c.close()
    conn.close()

In [4]:
# Sqlite db file will continute to grow in size after any deletes or drops
# This will optimize the database file and reduce its size
# You can also just remove the db file and recreate.
# http://www.sqlitetutorial.net/sqlite-vacuum/
conn = sqlite3.connect("twitterTesla.db")
conn.execute("VACUUM")

In [5]:
conn = sqlite3.connect("teslaTwitter.db")
c = conn.cursor()

# load stock into sqlite table
filepath = 'csv/tslaquotes.csv'

# read csv file
stock_df = pd.read_csv(filepath, parse_dates=['date'], usecols=['date', 'close', 'open', 'high', 'low'])

# append the data to the 'tesla' table in the 'teslaTwitter.db'
stock_df.to_sql("tesla", conn, if_exists = "append", index=False)

In [6]:
conn = sqlite3.connect("teslaTwitter.db")
c = conn.cursor()

# Pull out all of the json and put it into a tweets array to iterate through and create the data frame with
with open(os.path.join('json/', 'musktweets.json')) as json_file:
    tweets = []
    for line in json_file:
        tweets.append(json.loads(line))

# Examples of how to access JSON attributes of my tweets
# print(tweets[0]['created_at'])
# print(tweets[0]['full_text'])
# print(tweets[0]['id_str'])

tweet_df = pd.DataFrame(tweets, columns=['id_str','created_at','full_text'])

tweet_df.to_sql("tweets", conn, if_exists="append", index=False)

In [7]:
print(tweet_df[:1])
print("---------------")
print(stock_df[:1])

# TO DO: CONVERT DATE TIME FORMAT OF STOCK PRICES TO MATCH EACH OTHER
# WRITE THE SQL COMMAND TO THEN PULL TWEETS AND PRICES OF THE SAME DATE
# GRAPH THIS SQL TABLE

                id_str                      created_at  \
0  1039588735157850113  Tue Sep 11 18:57:23 +0000 2018   

                                           full_text  
0  RT @NASAJPL: #NeverForget\nThere is a 9/11 tri...  
---------------
                 date   close    open    high    low
0 2018-11-06 16:00:00  294.31  263.87  297.93  262.1
